In [14]:
pip install cvxpy


In [15]:
pip install numpy


In [59]:
import numpy as np
import cvxpy as cp

def  find_decomposition(budget, preferences):
    n = len(preferences)  # Number of citizens
    m = len(budget)  # Number of issues
    C = sum(budget)  # Total budget

    # Convert preferences to a matrix form for easier manipulation
    pref_matrix = np.zeros((n, m))
    for i, prefs in enumerate(preferences):
        for j in prefs:
            pref_matrix[i, j] = 1

    # Each citizen's share of the budget
    share_per_citizen = C / n

    # Setup the optimization variables
    d = cp.Variable((n, m))

    # The utility function to maximize: sum of logs of allocated budgets to supported issues
    utilities = []
    for i in range(n):
        for j in range(m):
            utilities.append(cp.log(d[i, j] + 1) * pref_matrix[i, j])  # Adding 1 to avoid log(0)

    # Objective function: maximize the sum of utilities
    objective = cp.Maximize(cp.sum(utilities))

    # Constraints
    constraints = []

    # Each issue gets its exact budget
    for j in range(m):
        constraints.append(cp.sum(d[:, j]) == budget[j])

    # Each citizen uses up their share of the budget, considering only the issues they care about
    for i in range(n):
        constraints.append(cp.sum(cp.multiply(d[i, :], pref_matrix[i, :])) == share_per_citizen)

    # d[i, j] >= 0 for all i, j
    constraints += [d >= 0]

    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve()

    # Check if the problem was solved successfully
    if problem.status not in ["infeasible", "unbounded"]:
        # If solved, prepare a more readable output
        allocation_matrix = np.round(d.value).astype(int)  # Ensure it's rounded and converted to integers
        readable_output = ""
        for i in range(n):
            for j in range(m):
                if allocation_matrix[i, j] > 0:
                    readable_output += f"Player {chr(65 + i)} contributes {allocation_matrix[i, j]} to issue {j}; "
        return readable_output.strip('; ')
    else:
        return "No feasible breakdown."




In [60]:

    # Example usage
    print("Example 1: ")
    budget = [400, 50, 50, 0]  # Budget
    preferences = [{0, 1}, {0, 2}, {0, 3}, {1, 2}, {0}]  # Preferences
    breakdown =  find_decomposition(budget, preferences)
    if breakdown is not None and "No feasible breakdown." not in breakdown:
        print("Break down found:\n", breakdown)
    else:
        print("No feasible breakdown.")


    print("\nExample 2 No feasible breakdown.: ")
    budget_2 = [0, 432]  # Budget
    preferences_2 = [{0}, {1}, {1}]  # Preferences
    breakdown_2 =  find_decomposition(budget_2, preferences_2)
    if breakdown_2 is not None and "No feasible breakdown." not in breakdown_2:
        print("Break down found:\n", breakdown_2)
    else:
        print("No feasible breakdown\n")


Example 1: 
Break down found:
 Player A contributes 100 to issue 0; Player B contributes 100 to issue 0; Player C contributes 100 to issue 0; Player D contributes 50 to issue 1; Player D contributes 50 to issue 2; Player E contributes 100 to issue 0

Example 2 No feasible breakdown.: 
No feasible breakdown

